# 動的計画法

## 導入

- 動的計画法とは
  - 最適化計算手法の一種
- 最適化計算とは
  - 関数の最大値or最小値を求める計算
  - 一般には全組合せの調査必要
- 関数が対ごとの2変数関数の和の場合
  - 最適解の効率的な解法 ＝ 動的計画法

## 方法

\begin{equation*}
J  = f_1(x_1) + h_1(x_1, x_2) + h_2(x_2, x_3) + \cdots + h_{n-1}(x_{n-1}, x_n) \rightarrow \max
\end{equation*}

- $x_1$に着目
  - 第1項、第2項のみ
- 可能な全ての$x_2$に対し最適な$x_1$を計算
- 第1項、第2項は次式に書ける

\begin{equation*}
f_2(x_2)  = \max_{x_1}[f_1(x_1) + h_1(x_1, x_2)]
\end{equation*}

- 元の式を$f_2$を使って表現

\begin{equation*}
J  = f_2(x_2) + h_2(x_2, x_3) + \cdots + h_{n-1}(x_{n-1}, x_n) \rightarrow \max
\end{equation*}

- $x_2$, $x_3$と繰り返す
- $x_n$まで計算すると最適な$\{x_1, x_2, ..., x_n\}$が得られる

## カエル跳び問題

### 問題

$N$個の足場があって、$i$番目の足場の高さは$h_i$です。
最初、足場1にカエルがいて、ぴょんぴょん跳ねながら足場$N$へと向かいます。カエルは足場$i$にいるときに

- 足場$i$から足場$i+1$へと移動する (そのコストは$|h_i-h_{i+1}$)
- 足場$i$から足場$i+2$へと移動する (そのコストは$|h_i-h_{i+2}$)

のいずれかの行動を選べます。カエルが足場1から足場$N$へと移動するのに必要な最小コストを求めよ。

<img width="500" src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F182963%2Fc010da1b-e2d0-3121-9050-5d306dfe3c41.jpeg?ixlib=rb-4.0.0&auto=format&gif-q=60&q=75&w=1400&fit=max&s=1c8e3db3816ce58e329c8f08fbeb5a79">

In [2]:
import numpy as np
def Frog1(footholds):
    one_diffs = [abs(f0 - f1) for f0, f1 in zip(footholds, footholds[1:])]
    two_diffs = [abs(f0 - f2) for f0, f2 in zip(footholds, footholds[2:])]
    indexes = [[0]]
    scores = np.zeros((1,))
    indexes.append(indexes[-1] + [len(scores)])
    scores = np.append(scores, one_diffs.pop(0))
    for diff in zip(two_diffs, one_diffs):
        score = np.array([scores[-2] + diff[-2], scores[-1] + diff[-1]])
        i = np.argmin(score) - 2
        indexes.append(indexes[i] + [len(scores)])
        scores = np.append(scores, score[i])
    return indexes[-1]

In [13]:
footholds = [2, 9, 4, 5, 1, 6]
Frog1(footholds)

[0, 2, 3, 5]

## ナップサック問題
https://qiita.com/drken/items/a5e6fe22863b7992efdb

- $n$個の荷物
- 荷物$i$の重さ$w_i$, 価値$v_i$
- 重さの合計がUを越えない様に荷物を選択
- 価値の合計の最大値を求めよ

In [58]:
import numpy as np
def knapsack(weights, values, max_weight):
    num_things = len(weights)
    dp = np.zeros((num_things + 1, max_weight + 1))
    indexes = dict()
    #
    for i, (weight, value) in enumerate(zip(weights, values)):
        for u in range(max_weight + 1):
            if u >= weight:
                if dp[i, u - weight] + value > dp[i, u]:
                    dp[i + 1, u] = dp[i, u - weight] + value
                    indexes[(i + 1, u)] = indexes.get((i, u - weight), []) + [i]
                    continue
            dp[i + 1, u] = dp[i, u]
    return dp[num_things, max_weight], indexes[(num_things, max_weight)]

In [59]:
weights = [2, 1, 3, 2, 1, 5]
values = [3, 2, 6, 1, 3, 85]
max_weight = 9
knapsack(weights, values, max_weight)

(94.0, [4, 5])

## 部分和問題
$n$個の正の整数$a[0], $a[1], ..., a[n-1]$と正の整数$A$が与えられる。
これらの整数から何個かの整数を選んで総和が$A$になるようにすることが可能か判定せよ。
可能ならば "YES" と出力し、不可能ならば "NO" と出力せよ。

【制約】
- $1\le n\le 100$
- $1\le a[i]\le 1000$
- $1\le A\le 10000$

【数値例】

1. 
   - $n=3$
   - $a=(7, 5, 3)$
   - $A=10$
   - 答え: YES (7 と 3 を選べばよいです)
2.
   - $n=2$
   - $a=(9, 7)$
   - $A=6$
   - 答え: NO

In [6]:
import numpy as np
def partialsum(array, sum):
    n = array.size
    dp = np.zeros((n + 1, sum + 1)).astype(bool)
    dp[0, 0] = True

    for i in range(n):
        for j in range(sum + 1):
            dp[i+1, j] |= dp[i, j]
            if j >= array[i]:
                dp[i+1, j] |= dp[i, j-array[i]]

    return dp[n, sum]

In [7]:
array = np.array([7, 5, 3])
sum = 10
partialsum(array, sum)

True

In [8]:
array = np.array([9, 7])
sum = 6
partialsum(array, sum)

False

## 部分和数え上げ問題
$n$個の正の整数$a[0], a[1], ..., a[n-1]$と正の整数$A$が与えられる。
これらの整数から何個かの整数を選んで総和が$A$になるようにする方法が何通りあるかを求めよ。
ただし、答えがとても大きくなる可能性があるので、1,000,000,009 で割った余りで出力せよ。

【制約】
- $1\le n\le 100$
- $1\le a[i]\le 1000$
- $1\le A\le 10000$

【数値例】

1. 
   - $n=5$
   - $a=(7, 5, 3, 1, 8)$
   - $A=12$
   - 答え: 2 ((7と5), (3と1と8) の 2 通りがあります)
2.
   - $n=4$
   - $a=(4, 1, 1, 1)$
   - $A=5$
   - 答え: 3 ((4, 1), (4, 1), (4, 1) の 3 通りがあります。同じ 1 でも index が違うものは異なる組合わせとみなします)

In [5]:
import numpy as np
def partialsum_count(array, sum):
    MOD = 1000000009
    n = array.size
    dp = np.zeros((n + 1, sum + 1)).astype(int)
    dp[0, 0] = 1

    for i in range(n):
        for j in range(sum + 1):
            dp[i+1, j] += dp[i, j]
            dp[i+1, j] %= MOD
            if j >= array[i]:
                dp[i+1, j] += dp[i, j-array[i]]
                dp[i+1, j] %= MOD

    return dp[n, sum]

In [6]:
array = np.array([7, 5, 3, 1, 8])
sum = 12
partialsum_count(array, sum)

2

In [7]:
array = np.array([4, 1, 1, 1])
sum = 5
partialsum_count(array, sum)

3

## 最小個数部分和問題
$n$個の正の整数$a[0], a[1], ..., a[n-1]$と正の整数$A$が与えられる。
これらの整数から何個かの整数を選んで総和が$A$にする方法をすべて考えた時、選ぶ整数の個数の最小値を求めよ。
$A$にすることができない場合は$-1$と出力せよ。

【制約】
- $1\le n\le 100$
- $1\le a[i]\le 1000$
- $1\le A\le 10000$

【数値例】

1. 
   - $n=5$
   - $a=(7, 5, 3, 1, 8)$
   - $A=12$
   - 答え: 2 ((7, 5)と(3, 1, 8)とがありますが、(7, 5) の2個が最小です)
2.
   - $n=2$
   - $a=(7, 5)$
   - $A=6$
   - 答え: -1

In [13]:
import numpy as np
def partialsum_min_n(array, sum):
    big = 1000000
    n = array.size
    dp = np.ones((n + 1, sum + 1)).astype(int) * big
    dp[0, 0] = 0

    for i in range(n):
        for j in range(sum + 1):
            dp[i+1, j] = dp[i, j]
            if j >= array[i]:
                dp[i+1, j] = min(dp[i, j-array[i]] + 1, dp[i, j])

    return dp[n, sum] if dp[n, sum] < big else -1

In [14]:
array = np.array([7, 5, 3, 1, 8])
sum = 12
partialsum_min_n(array, sum)

2

In [15]:
array = np.array([7, 5])
sum = 6
partialsum_min_n(array, sum)

-1

## K個以内部分和問題
$n$個の正の整数$a[0], a[1], ..., a[n-1]$と正の整数$A$が与えられる。
これらの整数から$k$個以内の整数を選んで総和が$A$になるようにすることが可能か判定せよ。
可能ならば "YES" と出力し、不可能ならば "NO" と出力せよ。

【制約】
- $1\le n\le 100$
- $1\le a[i]\le 1000$
- $1\le A\le 10000$

【数値例】

1. 
   - $n=3$
   - $K=2$
   - $a=(7, 5, 3)$
   - $A=10$
   - 答え: YES (7 と 3 を選べばよいです)
2.
   - $n=3$
   - $K=1$
   - $a=(7, 5, 3)$
   - $A=10$
   - 答え: NO

In [16]:
import numpy as np
def partialsum_le_k(array, sum, le_k):
    big = 1000000
    n = array.size
    dp = np.ones((n + 1, sum + 1)).astype(int) * big
    dp[0, 0] = 0

    for i in range(n):
        for j in range(sum + 1):
            dp[i+1, j] = dp[i, j]
            if j >= array[i]:
                dp[i+1, j] = min(dp[i, j-array[i]] + 1, dp[i, j])

    return dp[n, sum] <= le_k

In [17]:
array = np.array([7, 5, 3])
sum = 10
le_k = 2
partialsum_le_k(array, sum, le_k)

True

In [18]:
array = np.array([7, 5, 3])
sum = 10
le_k = 1
partialsum_le_k(array, sum, le_k)

False